In [1]:
import osmnx as ox
ox.config(log_file=True, log_console=True, use_cache=True)
ox.__version__

import networkx as nx

import pandas as pd
from pandas import DataFrame

import pandana, matplotlib.pyplot as plt
%matplotlib inline
from pandana.loaders import osm

from shapely.geometry import Point

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import xlsxwriter
import openpyxl
import osmnx as ox
ox.config(log_file=True, log_console=True, use_cache=True)
ox.__version__

'0.10'

In [141]:
#####Origincal point
#####Plaza Luceros: 38.3459813, -0.49069405
#####Plaza Gabriel Miró:  38.343352, -0.484987
point = (38.343352, -0.484987)
dist = 500

In [10]:
#####Placement effect: move original point to south-middle point
#####Plaza Luceros: 38.35048314998173, -0.49069405
#####Plaza Gabriel Miró: 38.34785383227921, -0.484987
point = (38.34785383227921, -0.484987)
dist = 2000

In [151]:
##### lat & lon 
bbox = ox.bbox_from_point(point, distance = dist)
north, south, east, west = ox.bbox_from_point(point, dist, project_utm=False) #If lat & lon are desired : project_utm=Fale
north, south, east, west

(38.356857485910105,
 38.33885016264137,
 -0.4735539844519557,
 -0.49642015458566685)

In [45]:
##### X & Y
bbox = ox.bbox_from_point(point, distance = dist)
north, south, east, west = ox.bbox_from_point(point, dist, project_utm=True) #If lat & lon are desired : project_utm=Fale
north, south,east, west

(4250183.352319879, 4245183.352319879, 721773.9977197393, 716773.9977197393)

In [11]:
networkType = 'walk'

G_walk = ox.graph_from_point(point, distance=dist, network_type=networkType)
#fig, ax = ox.plot_graph(ox.project_graph(G), node_size=0, show=False, close=False)
# fig, ax = ox.plot_graph(G_walk, show=False, close=False) #We’ll just take advantage of the show and close parameters in OSMnx to prevent it from finishing the Matplotlib operation and instead returning the unclosed fix and ax objects.

#####Covert stats results into dataframe and save as excel
statsBasicWalk = ox.stats.basic_stats(G_walk, area=dist*dist, clean_intersects=True, tolerance=15, circuity_dist='euclidean')
dfStatsBasicWalk = pd.DataFrame.from_dict(statsBasicWalk) 
statsExtendedWalk = ox.extended_stats(G_walk, connectivity=True, anc=False, ecc=False, bc=True, cc=True)
dfStatsExtendedWalk = pd.DataFrame.from_dict(statsExtendedWalk)

#####Create a dictionary combines basic and extended stats
# statsWalk = ox.basic_stats(G_walk, area=dist*dist, clean_intersects=True, tolerance=15, circuity_dist='euclidean')
# extendedStatsWalk = ox.extended_stats(G_walk,connectivity=True, anc=False, ecc=False, bc=True, cc=True)
# for key, value in extendedStatsWalk.items():
#     statsWalk[key] = value
# pd.Series(statsWalk)

#####Horizontal merge basic and extended stats of walk network
##### https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
dfStatsMergeWalk  = pd.merge(dfStatsBasicWalk,                  #data frame 1
                              dfStatsExtendedWalk,                   #data frame 2
                              left_on=None, #column in data frame 1
                              right_on=None,         #column in data frame 2
                              left_index=True, right_index=True, 
                              how='outer',              
                              sort=False)
#                             suffixes=['_from', '_to'])
dfStatsMergeWalk

,n,m,k_avg,intersection_count,streets_per_node_avg,streets_per_node_counts,streets_per_node_proportion,edge_length_total,edge_length_avg,street_length_total,...,pagerank_max_node,pagerank_max,pagerank_min_node,pagerank_min,node_connectivity,edge_connectivity,closeness_centrality,closeness_centrality_avg,betweenness_centrality,betweenness_centrality_avg
0,4402.0,13962.0,6.34348,4155.0,3.200818,0.0,0.000000,722457.968,51.74459,363546.631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4402.0,13962.0,6.34348,4155.0,3.200818,247.0,0.056111,722457.968,51.74459,363546.631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4402.0,13962.0,6.34348,4155.0,3.200818,0.0,0.000000,722457.968,51.74459,363546.631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4402.0,13962.0,6.34348,4155.0,3.200818,2808.0,0.637892,722457.968,51.74459,363546.631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4402.0,13962.0,6.34348,4155.0,3.200818,1319.0,0.299637,722457.968,51.74459,363546.631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4402.0,13962.0,6.34348,4155.0,3.200818,25.0,0.005679,722457.968,51.74459,363546.631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4402.0,13962.0,6.34348,4155.0,3.200818,3.0,0.000682,722457.968,51.74459,363546.631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13843560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.748126e+09,0.000955,3.782586e+09,0.000038,1.0,1.0,0.000370,0.000544,0.009702,0.007042
13843562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.748126e+09,0.000955,3.782586e+09,0.000038,1.0,1.0,0.000338,0.000544,0.004725,0.007042
21323845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.748126e+09,0.000955,3.782586e+09,0.000038,1.0,1.0,0.000354,0.000544,0.004258,0.007042


In [12]:
networkType = 'bike'

G_bike = ox.graph_from_point(point, distance=dist, network_type=networkType)
#fig, ax = ox.plot_graph(ox.project_graph(G), node_size=0, show=False, close=False)
# fig, ax = ox.plot_graph(G_bike, show=False, close=False) #We’ll just take advantage of the show and close parameters in OSMnx to prevent it from finishing the Matplotlib operation and instead returning the unclosed fix and ax objects.

#####Covert stats results into dataframe and save as excel
statsBasicBike = ox.stats.basic_stats(G_bike, area=dist*dist, clean_intersects=True, tolerance=15, circuity_dist='euclidean')
dfStatsBasicBike = pd.DataFrame.from_dict(statsBasicBike)  
statsExtendedBike = ox.extended_stats(G_bike, connectivity=True, anc=False, ecc=False, bc=True, cc=True)
dfStatsExtendedBike = pd.DataFrame.from_dict(statsExtendedBike)

#####Create a dictionary combines basic and extended stats
# statsBike = ox.basic_stats(G_bike, area=dist*dist, clean_intersects=True, tolerance=15, circuity_dist='euclidean')
# extendedStatsBike = ox.extended_stats(G_bike,connectivity=True, anc=False, ecc=False, bc=True, cc=True)
# for key, value in extendedStatsBike.items():
#     statsBike[key] = value
# pd.Series(statsBike)

#####Horizontal merge basic and extended stats bike network
##### https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
dfStatsMergeBike  = pd.merge(dfStatsBasicBike,                  #data frame 1
                              dfStatsExtendedBike,                   #data frame 2
                              left_on=None, #column in data frame 1
                              right_on=None,         #column in data frame 2
                              left_index=True, right_index=True, 
                              how='outer',              
                              sort=False)
#                             suffixes=['_from', '_to'])
dfStatsMergeBike

,n,m,k_avg,intersection_count,streets_per_node_avg,streets_per_node_counts,streets_per_node_proportion,edge_length_total,edge_length_avg,street_length_total,...,pagerank_max_node,pagerank_max,pagerank_min_node,pagerank_min,node_connectivity,edge_connectivity,closeness_centrality,closeness_centrality_avg,betweenness_centrality,betweenness_centrality_avg
0,3312.0,6628.0,4.002415,3126.0,3.214372,0.0,0.000000,406238.979,61.291337,316445.601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3312.0,6628.0,4.002415,3126.0,3.214372,186.0,0.056159,406238.979,61.291337,316445.601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3312.0,6628.0,4.002415,3126.0,3.214372,44.0,0.013285,406238.979,61.291337,316445.601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3312.0,6628.0,4.002415,3126.0,3.214372,1984.0,0.599034,406238.979,61.291337,316445.601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3312.0,6628.0,4.002415,3126.0,3.214372,1074.0,0.324275,406238.979,61.291337,316445.601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3312.0,6628.0,4.002415,3126.0,3.214372,20.0,0.006039,406238.979,61.291337,316445.601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3312.0,6628.0,4.002415,3126.0,3.214372,4.0,0.001208,406238.979,61.291337,316445.601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13843558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.643724e+09,0.001353,1.240662e+09,0.000046,1.0,1.0,0.000418,0.000432,0.072149,0.010377
13843560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.643724e+09,0.001353,1.240662e+09,0.000046,1.0,1.0,0.000356,0.000432,0.010799,0.010377
13843562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.643724e+09,0.001353,1.240662e+09,0.000046,1.0,1.0,0.000316,0.000432,0.015612,0.010377


In [13]:
networkType = 'drive'

G_drive = ox.graph_from_point(point, distance=dist, network_type=networkType)
#fig, ax = ox.plot_graph(ox.project_graph(G), node_size=0, show=False, close=False)
# fig, ax = ox.plot_graph(G_drive, show=False, close=False) #We’ll just take advantage of the show and close parameters in OSMnx to prevent it from finishing the Matplotlib operation and instead returning the unclosed fix and ax objects.

#####Covert stats results into dataframe and save as excel
statsBasicDrive = ox.stats.basic_stats(G_drive, area=dist*dist, clean_intersects=True, tolerance=15, circuity_dist='euclidean')
dfStatsBasicDrive = pd.DataFrame.from_dict(statsBasicDrive) 
statsExtendedDrive = ox.extended_stats(G_drive, connectivity=True, anc=False, ecc=False, bc=True, cc=True)
dfStatsExtendedDrive = pd.DataFrame.from_dict(statsExtendedDrive)

#####Create a dictionary combines basic and extended stats
# statsDrive = ox.basic_stats(G_drivearea=dist*dist, clean_intersects=True, tolerance=15, circuity_dist='euclidean')
# extendedStatsDrive = ox.extended_stats(G_drive,connectivity=True, anc=False, ecc=False, bc=True, cc=True)
# for key, value in extendedStatsDrive.items():
#     statsDrive[key] = value
# pd.Series(statsDrive)

#####Horizontal merge basic and extended stats of drive network
##### https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
dfStatsMergeDrive  = pd.merge(dfStatsBasicDrive,                  #data frame 1
                              dfStatsExtendedDrive,                   #data frame 2
                              left_on=None, #column in data frame 1
                              right_on=None,         #column in data frame 2
                              left_index=True, right_index=True, 
                              how='outer',              
                              sort=False)
#                             suffixes=['_from', '_to'])
dfStatsMergeDrive

,n,m,k_avg,intersection_count,streets_per_node_avg,streets_per_node_counts,streets_per_node_proportion,edge_length_total,edge_length_avg,street_length_total,...,pagerank_max_node,pagerank_max,pagerank_min_node,pagerank_min,node_connectivity,edge_connectivity,closeness_centrality,closeness_centrality_avg,betweenness_centrality,betweenness_centrality_avg
0,2054.0,3670.0,3.573515,1977.0,3.236611,0.0,0.000000,268058.334,73.040418,232761.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2054.0,3670.0,3.573515,1977.0,3.236611,77.0,0.037488,268058.334,73.040418,232761.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2054.0,3670.0,3.573515,1977.0,3.236611,27.0,0.013145,268058.334,73.040418,232761.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2054.0,3670.0,3.573515,1977.0,3.236611,1302.0,0.633885,268058.334,73.040418,232761.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2054.0,3670.0,3.573515,1977.0,3.236611,632.0,0.307692,268058.334,73.040418,232761.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2054.0,3670.0,3.573515,1977.0,3.236611,13.0,0.006329,268058.334,73.040418,232761.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2054.0,3670.0,3.573515,1977.0,3.236611,3.0,0.001461,268058.334,73.040418,232761.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13843560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.487187e+09,0.002333,1.675134e+09,0.000075,1.0,1.0,0.000364,0.000422,0.014182,0.014445
13843562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.487187e+09,0.002333,1.675134e+09,0.000075,1.0,1.0,0.000318,0.000422,0.016994,0.014445
21323845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.487187e+09,0.002333,1.675134e+09,0.000075,1.0,1.0,0.000296,0.000422,0.007186,0.014445


In [14]:
#####Vertical combine 3 dataframes : walk, bike and drive 
##### https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

frames = [dfStatsMergeWalk, dfStatsMergeBike, dfStatsMergeDrive]
dfStatsConcat = pd.concat(frames, keys=['walk', 'bike', 'drive'])
dfStatsConcat

n        m    k_avg  intersection_count  \
walk  0           4402.0  13962.0  6.34348              4155.0   
      1           4402.0  13962.0  6.34348              4155.0   
      2           4402.0  13962.0  6.34348              4155.0   
      3           4402.0  13962.0  6.34348              4155.0   
      4           4402.0  13962.0  6.34348              4155.0   
...                  ...      ...      ...                 ...   
drive 6771884962     NaN      NaN      NaN                 NaN   
      6771884966     NaN      NaN      NaN                 NaN   
      6978104416     NaN      NaN      NaN                 NaN   
      7282048458     NaN      NaN      NaN                 NaN   
      7282048459     NaN      NaN      NaN                 NaN   

                  streets_per_node_avg  streets_per_node_counts  \
walk  0                       3.200818                      0.0   
      1                       3.200818                    247.0   
      2                       3.200818                      0.0   
      3                       3.200818                   2808.0   
      4                       3.200818                   1319.0   
...                                ...                      ...   
drive 6771884962                   NaN                      NaN   
      6771884966                   NaN                      NaN   
      6978104416                   NaN                      NaN   
      7282048458                   NaN                      NaN   
      7282048459                   NaN                      NaN   

                  streets_per_node_proportion  edge_length_total  \
walk  0                              0.000000         722457.968   
      1                              0.056111         722457.968   
      2                              0.000000         722457.968   
      3                              0.637892         722457.968   
      4                              0.299637         722457.968   
...                                       ...                ...   
drive 6771884962                          NaN                NaN   
      6771884966                          NaN                NaN   
      6978104416                          NaN                NaN   
      7282048458                          NaN                NaN   
      7282048459                          NaN                NaN   

                  edge_length_avg  street_length_total  ...  \
walk  0                  51.74459           363546.631  ...   
      1                  51.74459           363546.631  ...   
      2                  51.74459           363546.631  ...   
      3                  51.74459           363546.631  ...   
      4                  51.74459           363546.631  ...   
...                           ...                  ...  ...   
drive 6771884962              NaN                  NaN  ...   
      6771884966              NaN                  NaN  ...   
      6978104416              NaN                  NaN  ...   
      7282048458              NaN                  NaN  ...   
      7282048459              NaN                  NaN  ...   

                  pagerank_max_node  pagerank_max  pagerank_min_node  \
walk  0                         NaN           NaN                NaN   
      1                         NaN           NaN                NaN   
      2                         NaN           NaN                NaN   
      3                         NaN           NaN                NaN   
      4                         NaN           NaN                NaN   
...                             ...           ...                ...   
drive 6771884962       1.487187e+09      0.002333       1.675134e+09   
      6771884966       1.487187e+09      0.002333       1.675134e+09   
      6978104416       1.487187e+09      0.002333       1.675134e+09   
      7282048458       1.487187e+09      0.002333       1.675134e+09   
      7282048459       1.487187e+09      0.002333       1.6

In [15]:
writer = pd.ExcelWriter('D:/Thesis/p2_plazaGabrielMiro/placementEffect/walk, bike, drive/2000m.xlsx', engine='xlsxwriter')

dfStatsConcat.to_excel(writer, sheet_name='Sheet1', startcol=0, startrow=0,header = True, index = True )
# df_stats_extended.to_excel(writer, sheet_name='Sheet1', startcol=22, startrow=0, header = True, index = True)

writer.save()

# Weighted edge

In [196]:
#####Origincal point
#####Plaza Luceros: 38.3459813, -0.49069405
#####Plaza Gabriel Miró:  38.343352, -0.484987
point = (38.343352, -0.484987)
dist = 2500

In [181]:
#####Placement effect: move original point to south-middle point
#####Plaza Luceros: 38.35048314998173, -0.49069405
#####Plaza Gabriel Miró: 38.34785383227921, -0.484987
point = (38.34785383227921, -0.484987)
dist = 2500

In [123]:
bbox = ox.bbox_from_point(point, distance = 
                          dist)
north, south, east, west = ox.bbox_from_point(point, dist, project_utm=False) #If lat & lon are desired : project_utm=Fale
north, south, east, west

(38.34785383227921,
 38.33885016371925,
 -0.4792708286348258,
 -0.49070320611838064)

In [197]:
G = ox.graph_from_point(point, distance=dist, 
                       infrastructure='way["highway"~"footway"]',
#                       infrastructure='way["highway"~"pedestrian"]',
#                       infrastructure='way["highway"~"tertiary"]',
#                       infrastructure='way["highway"~"cycleway"]',
#                       infrastructure='way["highway"~"path"]', #no data
#                       infrastructure='way["highway"~"steps"]',
#                       infrastructure='way["highway"~"track"]',
#                       infrastructure='way["highway"~"corridor"]',#no data
#                       infrastructure='way["highway"~"elevator"]', #no data
#                       infrastructure='way["highway"~"escalator"]',#no data
#                       infrastructure='way["highway"~"proposed"]',#no data
#                       infrastructure='way["highway"~"construction"]',#no data
#                       infrastructure='way["highway"~"bridleway"]',#no data
#                       infrastructure='way["highway"~"abandoned"]',#no data
#                       infrastructure='way["highway"~"platform"]',#no data
#                       infrastructure='way["highway"~"raceway"]',#no data
#                       infrastructure='way["highway"~"living_street"]', #NetworkXPointlessConcept: Connectivity is undefined for the null graph.
#                       infrastructure='way["highway"~"residental"]',  #No data                    
                       )
statsBasic = ox.stats.basic_stats(G, area=dist*dist, clean_intersects=True, tolerance=15, circuity_dist='euclidean')
dfStatsBasic = pd.DataFrame.from_dict(statsBasic) 
statsExtended = ox.extended_stats(G, connectivity=True, anc=False, ecc=False, bc=True, cc=True)
dfStatsExtended = pd.DataFrame.from_dict(statsExtended)

# dfStatsMergeFootway  = pd.merge(dfStatsBasic,                  #data frame 1
#                               dfStatsExtended,                   #data frame 2
#                               left_on=None, #column in data frame 1
#                               right_on=None,         #column in data frame 2
#                               left_index=True, right_index=True, 
#                               how='outer',              
#                               sort=False)
# #                             suffixes=['_from', '_to'])
# dfStatsMergeFootway

dfStatsMergePedestrian  = pd.merge(dfStatsBasic,                  #data frame 1
                              dfStatsExtended,                   #data frame 2
                              left_on=None, #column in data frame 1
                              right_on=None,         #column in data frame 2
                              left_index=True, right_index=True, 
                              how='outer',              
                              sort=False)
#                             suffixes=['_from', '_to'])
dfStatsMergePedestrian

,n,m,k_avg,intersection_count,streets_per_node_avg,streets_per_node_counts,streets_per_node_proportion,edge_length_total,edge_length_avg,street_length_total,...,pagerank_max_node,pagerank_max,pagerank_min_node,pagerank_min,node_connectivity,edge_connectivity,closeness_centrality,closeness_centrality_avg,betweenness_centrality,betweenness_centrality_avg
0,680.0,1886.0,5.547059,589.0,2.792647,0.0,0.000000,90124.17,47.78588,45162.043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,680.0,1886.0,5.547059,589.0,2.792647,91.0,0.133824,90124.17,47.78588,45162.043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,680.0,1886.0,5.547059,589.0,2.792647,0.0,0.000000,90124.17,47.78588,45162.043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,680.0,1886.0,5.547059,589.0,2.792647,549.0,0.807353,90124.17,47.78588,45162.043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,680.0,1886.0,5.547059,589.0,2.792647,39.0,0.057353,90124.17,47.78588,45162.043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,680.0,1886.0,5.547059,589.0,2.792647,1.0,0.001471,90124.17,47.78588,45162.043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219236029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.748126e+09,0.005254,4.747820e+09,0.000242,1.0,1.0,0.000776,0.000882,0.000000,0.033707
224831057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.748126e+09,0.005254,4.747820e+09,0.000242,1.0,1.0,0.000876,0.000882,0.000000,0.033707
243893700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.748126e+09,0.005254,4.747820e+09,0.000242,1.0,1.0,0.000793,0.000882,0.000000,0.033707
243895022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.748126e+09,0.005254,4.747820e+09,0.000242,1.0,1.0,0.000943,0.000882,0.000000,0.033707


In [ ]:
frames = [dfStatsMergeFootway, dfStatsMergePedestrian]
dfStatsConcat = pd.concat(frames, keys=['footway', 'pedestrian'])

In [198]:
writer = pd.ExcelWriter('D:/Thesis/p2_plazaGabrielMiro/originalPoint/weightedEdges/2500m.xlsx', engine='xlsxwriter')

dfStatsMergePedestrian.to_excel(writer, sheet_name='Sheet1', startcol=0, startrow=0,header = True, index = True )
# df_stats_extended.to_excel(writer, sheet_name='Sheet1', startcol=22, startrow=0, header = True, index = True)

writer.save()

In [ ]:
#why there are 4 times more edges for pedestrians than for cars (cellE4)? 
#If a street has one car line and two side walks I would expect a value near to 2? 
#What counts as edge here?
#if a street can be used by pedestrians in theory - would they use it in reality? 
#Calle San Fernando has side walks of perhaps 50
#cm on both sides but endless car traffic - absolutely unpleasant to walk and I have hardly seen pedestrians there. What to do with such a street?
#But normally a street with car and pedestrian should be included, yes. On the other hand, the worst like that one should be excluded, if
#possible. But how to decide that?

In [ ]:
# https://github.com/gboeing/osmnx/blob/master/osmnx/downloader.py
# driving: filter out un-drivable roads, service roads, private ways, and
    
    # anything specifying motor=no. also filter out any non-service roads that
    # are tagged as providing parking, driveway, private, or emergency-access
    # services
    filters['drive'] = ('["area"!~"yes"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|corridor|'
                        'elevator|escalator|proposed|construction|bridleway|abandoned|platform|raceway|service"]'
                        '["motor_vehicle"!~"no"]["motorcar"!~"no"]{}'
                        '["service"!~"parking|parking_aisle|driveway|private|emergency_access"]').format(
        settings.default_access)

    # drive+service: allow ways tagged 'service' but filter out certain types of
    # service ways
    filters['drive_service'] = ('["area"!~"yes"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|corridor|'
                                'elevator|escalator|proposed|construction|bridleway|abandoned|platform|raceway"]'
                                '["motor_vehicle"!~"no"]["motorcar"!~"no"]{}'
                                '["service"!~"parking|parking_aisle|private|emergency_access"]').format(
        settings.default_access)

    # walking: filter out cycle ways, motor ways, private ways, and anything
    # specifying foot=no. allow service roads, permitting things like parking
    # lot lanes, alleys, etc that you *can* walk on even if they're not exactly
    # pleasant walks. some cycleways may allow pedestrians, but this filter ignores
    # such cycleways.
    filters['walk'] = ('["area"!~"yes"]["highway"!~"cycleway|motor|proposed|construction|abandoned|platform|raceway"]'
                       '["foot"!~"no"]["service"!~"private"]{}').format(settings.default_access)

    # biking: filter out foot ways, motor ways, private ways, and anything
    # specifying biking=no
    filters['bike'] = ('["area"!~"yes"]["highway"!~"footway|steps|corridor|elevator|escalator|motor|proposed|'
                       'construction|abandoned|platform|raceway"]'
                       '["bicycle"!~"no"]["service"!~"private"]{}').format(settings.default_access)

    # to download all ways, just filter out everything not currently in use or
    # that is private-access only
    filters['all'] = ('["area"!~"yes"]["highway"!~"proposed|construction|abandoned|platform|raceway"]'
                      '["service"!~"private"]{}').format(settings.default_access)

    # to download all ways, including private-access ones, just filter out
    # everything not currently in use
    filters['all_private'] = '["area"!~"yes"]["highway"!~"proposed|construction|abandoned|platform|raceway"]'


In [ ]:
#####https://gis.stackexchange.com/questions/340681/getting-pedestrian-network-with-osmnx
infrastructure = ('way["highway"~"footway|pedestrian|cycleway|path|steps|living_street|tertiary|unclassified|residental|service"]',)
custom_filter = ('["footway"~"sidewalk"]["foot"~"designated|yes"]',)